In [1]:
# Baseline model, doing logistic regression on the data
# Import pandas and sklearn

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv('train.csv')

# Split the data into training and testing data
# X = df.drop(['Exited', 'Surname', 'CustomerId', 'Gender', 'Geography'], axis=1)
X = df.drop(['Exited', 'Surname', 'CustomerId'], axis=1)

In [3]:
# one-hot encode the categorical variables (gender, geography)
enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(X).toarray()


In [4]:
# Describe the X dataframe
# pd.DataFrame(X).describe()

In [4]:
y = df['Exited']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

LogisticRegression()

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
